### Imports

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

In [3]:
import datetime
from sklearn.model_selection import StratifiedKFold, KFold
import math

In [4]:
from sklearn.decomposition import PCA

In [5]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

### Read Data

In [6]:
df_y_train_c = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')

In [7]:
df_x_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_x_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
df_y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv').age
IDtrain = pd.DataFrame({'custid': df_x_train.custid.unique()})
IDtest = pd.DataFrame({'custid': df_x_test.custid.unique()})

# log 데이터 전처리

In [8]:
tr = pd.concat([df_x_train, df_x_test])
tr['real_amt']= tr.tot_amt / tr.inst_mon
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100

In [9]:
id_data = pd.DataFrame({'custid': tr.custid.unique()})

In [10]:
tr['sales_hour'] = tr['sales_time']//100
tr['sales_min'] = tr['sales_time']%100

In [11]:
tr["sales_date"] = np.where(tr["sales_month"] >9 , 
                            (tr["sales_month"].astype(str)),("0"+tr["sales_month"].astype(str)))


In [12]:
tr["sales_date"] = np.where(tr["sales_day"]>9,(tr["sales_date"]+tr["sales_day"].astype(str)),
                            (tr["sales_date"]+"0"+tr["sales_day"].astype(str))
                            )

In [13]:
tr["sales_month"] = np.where(tr["sales_month"]>12 , tr["sales_month"]-12, tr["sales_month"])

In [14]:
tr["time"] = pd.to_datetime(tr.sales_time, format = "%H%M")

In [15]:
tr["refund"] = np.where(tr["net_amt"]<0,1,0)

In [16]:
tr['sales_month_ver1'] = tr['sales_month']
tr['sales_month'] = tr['sales_month'].apply(lambda x : x-12 if x > 12 else x )
tr['sales_hour'] = tr['sales_time'].apply(lambda x : x//100 )
tr['sales_sec'] = tr['sales_time'].apply(lambda x : x%100 )
tr['total_sec'] = tr['sales_time'].apply(lambda x : x//100*60 + x%100 )
tr['환불여부'] = tr['tot_amt'].apply(lambda x : 1 if x < 0 else 0 )
tr['refund'] = tr['tot_amt'].apply(lambda x : abs(x) if x < 0 else 0 )
tr['tot_amt'] = tr['tot_amt'] .apply(lambda x : 0 if x < 0 else x )
tr['real_amt'] = ( tr['tot_amt'] / tr['inst_mon'] ).apply(lambda x : math.trunc(x)) 
tr['sales_date'] =tr['sales_month_ver1'].astype(str).apply(lambda x : "0"+x if len(x) == 1  else x ) +tr['sales_day'].astype(str).apply(lambda x : "0"+x if len(x) == 1 else  x )
tr['sales_date'] = tr['sales_date'].astype(int)

In [17]:
df = pd.merge(tr, df_y_train_c, on = 'custid')
tr['age_group'] = df['age'].apply(lambda x : 'twenty' if (x>=20) & (x<30)
                                     else 'thirty' if (x>=30) & (x<40)
                                     else 'forty' if (x>=40) & (x<50)
                                     else 'fifty' if (x>=50) & (x<60)
                                     else 'sixty')

In [18]:
twenty_prefer_brd = tr[tr['age_group'] == 'twenty'].brd_nm.value_counts().index[1:].to_list()
thirty_prefer_brd = tr[tr['age_group'] == 'thirty'].brd_nm.value_counts().index[1:].to_list()
forty_prefer_brd = tr[tr['age_group'] == 'forty'].brd_nm.value_counts().index[1:].to_list()
fifty_prefer_brd = tr[tr['age_group'] == 'fifty'].brd_nm.value_counts().index[1:].to_list()
sixty_prefer_brd = tr[tr['age_group'] == 'sixty'].brd_nm.value_counts().index[1:].to_list()

def prefer_brd(x, list):
    for i in range(len(list)):
        if(x == list[i]):
            return len(list)-i

tr['20_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, twenty_prefer_brd)).fillna(0)
tr['30_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, thirty_prefer_brd)).fillna(0)
tr['40_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, forty_prefer_brd)).fillna(0)
tr['50_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, fifty_prefer_brd)).fillna(0)
tr['60_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, sixty_prefer_brd)).fillna(0)

In [19]:
tr["age_weight_sum"] = tr['20_weight']+tr['30_weight']+tr['40_weight']+tr['50_weight']+tr['60_weight']

In [20]:
tr["sales_season"] = tr['sales_month'].apply(lambda x : 'SPRING' if (x>=3) and (x<=5)
                                                  else 'SUMMER' if (x>=6) and (x<=8)
                                                  else 'FALL' if (x>=9) and (x<=11)
                                                  else 'WINTER')

In [21]:
tr['tot_amt>=0'] = tr['tot_amt'].apply(lambda x : x if x>=0 else 0)
tr['dis_amt>=0'] = tr['dis_amt'].apply(lambda x : x if x>=0 else 0)
tr['net_amt>=0'] = tr['net_amt'].apply(lambda x : x if x>=0 else 0)

# features 생성

In [22]:
features = []

In [23]:
from gensim.models import word2vec
import random

In [24]:
def oversample(x, n):
    lst = []
    random.seed(100)
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

# W2V goodcd

In [25]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
df['goodcd']=df['goodcd'].astype('str')
train_data=list(df.groupby('custid')['goodcd'].unique())

In [26]:
w2v_input = oversample(train_data, 5)

In [27]:
import gensim

In [28]:
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [29]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:02<00:00, 13983.21it/s]


In [30]:
w2v_good= pd.DataFrame(train_mean_vector)

# W2V brd_nm

In [31]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
train_data=list(df.groupby('custid')['brd_nm'].unique())

In [32]:
w2v_input = oversample(train_data, 5)
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [33]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:02<00:00, 15418.24it/s]


In [34]:
w2v_brd = pd.DataFrame(train_mean_vector)

# W2V corner_nm

In [35]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
train_data=list(df.groupby('custid')['corner_nm'].unique())

In [36]:
w2v_input = oversample(train_data, 5)
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [37]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:02<00:00, 16444.42it/s]


In [38]:
w2v_coner = pd.DataFrame(train_mean_vector)

- 환불데이터

In [39]:
t = tr.groupby("custid")["refund"].agg([("refund_count","sum")]).reset_index()
t["refund_bool"] = np.where(t["refund_count"]>0,1,0)
features.append(t)

- cn  구매하는 나이

In [40]:
data = pd.concat([id_data,df_y_train],axis = 1)
data = data.iloc[:len(df_y_train)]

In [41]:
tr_data = pd.merge(tr,data,on="custid")

In [42]:
tr_cor_mean = tr_data.groupby("corner_nm")["age"].mean()

In [43]:
tr_cor_mean # 주구매 코너를 이걸로 매핑해주자

corner_nm
DC캐주얼      37.875949
GBR  지원    46.125000
L/B침구      36.815166
N/B침구      37.051136
NB제화       36.269504
             ...    
홈데코        36.025105
홈쇼핑        62.000000
화장잡화       38.326142
화장품        31.930435
훼미닌부틱      48.281167
Name: age, Length: 308, dtype: float64

- brd 구매 평균나이

In [44]:
tr_brd_mean = tr_data.groupby("brd_nm")["age"].mean()

- pc 구매 평균나이

In [45]:
tr_pc_mean = tr_data.groupby("pc_nm")["age"].mean()

- 주구매 요일 나이평균

In [46]:
tr_day_mean = tr_data.groupby("sales_dayofweek")["age"].mean()

In [47]:
def refund_check(x):
    if x<0:
        return x
    else:
        return 0

- 환불 총액, 평균 , 최대

In [48]:
copy_df = tr.copy()
copy_df["tot_amt_refund"] = copy_df.net_amt.apply(refund_check)
t = copy_df.groupby("custid")["tot_amt_refund"].agg([("amt_refund","sum"),
                                                    ("amt_refund_mean","mean"),
                                                    ("amt_refund_max","min")]).reset_index()
features.append(t)

- 최애브랜드 사용수

In [49]:
def g(x):
    y = x.value_counts()
    y = y.iloc[0]
    return y

f = tr.groupby('custid')['brd_nm'].agg([('love_brd_count', g)]).reset_index()
f = f.fillna(0)
features.append(f)

- 연령대별 브랜드 선호도에 따른 가중치 비율

In [50]:
f = tr.groupby('custid')['20_weight', '30_weight', '40_weight', '50_weight', '60_weight', 'age_weight_sum'].sum()

f['20_weight_ratio'] = f['20_weight'] / f['age_weight_sum']
f['30_weight_ratio'] = f['30_weight'] / f['age_weight_sum']
f['40_weight_ratio'] = f['40_weight'] / f['age_weight_sum']
f['50_weight_ratio'] = f['50_weight'] / f['age_weight_sum']
f['60_weight_ratio'] = f['60_weight'] / f['age_weight_sum']

f = f.fillna(0)
features.append(f)

- 시간대별 방문횟수

In [51]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t)

- 시간대별 구매금액

In [52]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index().drop([0,1,8,9],axis = 1)
features.append(t)

- 날짜별 방문횟수

In [53]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()

In [54]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f)

- 날짜별 구매금액

In [55]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()

In [56]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f)

- 월간별 날짜횟수

In [57]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t)

- 월간 구매금액

In [58]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(t)

In [59]:
#쇼핑시간
time_sum = tr.groupby(['sales_date','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
time_sum['shopping_time'] = (time_sum['time']['max'] - time_sum['time']['min']).dt.total_seconds()
time_sum.drop(['sales_date','time'], axis=1, inplace=True,level=0)
time_sum = time_sum.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(time_sum)

In [60]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f)

In [61]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f)

In [62]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df)

In [63]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df)

In [64]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f)

In [65]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f)

In [66]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [67]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [68]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df)

In [69]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [70]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [71]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [72]:
#총구매액
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [73]:
#구매건수
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)

In [74]:
#평균구매가격

f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', "mean")]).reset_index()
features.append(f)

In [75]:
#평균할부개월수
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [76]:
#구매상품다양성
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f)

In [77]:
#수입상품_구매비율
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f)

In [78]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2)

In [79]:
#내점일수 

f = tr.groupby(by = 'custid')['sales_time'].agg([('내점일수','nunique')]).reset_index()
features.append(f)

In [80]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f)

In [81]:
#아동상품 구매건수
f = tr[tr.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features.append(f)

In [82]:
#상위 브랜드 40개의 구매빈도
top_brd = tr["brd_nm"].value_counts()[:40].index
df = tr.groupby(['custid','brd_nm'])["custid"].agg([('brd_count', 'count')]).reset_index()
df =df.pivot_table(values='brd_count', index=df.custid, columns="brd_nm", aggfunc='count',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df)

In [83]:
#상위 브랜드 40개의 구매금액
df = tr.groupby(['custid','brd_nm'])["tot_amt"].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns="brd_nm", aggfunc='sum',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df)

In [84]:
#주말방문비율
day = {'월' : 0 , '화' : 1 , '수' : 2 , '목' : 3 , '금' : 4 , '토': 5 , '일' : 6 }

tr['sales_dayofweek_num'] = tr['sales_dayofweek'].apply(lambda x : day[x] )

f = tr.groupby('custid')['sales_dayofweek_num'].agg([
    ('주말방문비율', lambda x: np.mean(x >4))]).reset_index()

features.append(f)

In [85]:
#계절별 구매 건수
def season(k):
    if 3 <= k <= 5 :
        return('봄')
    elif 6 <= k <= 8 :
        return('여름')
    elif 9 <= k <= 11 :    
        return('가을')
    else :
        return('겨울')
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [86]:
#계절별 구매 금액
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc="sum", fill_value=0).reset_index()
features.append(f)

In [87]:
#아침점심저녁별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침')
    elif 13 <= x <= 16 :
        return('점심')
    else :
        return('저녁')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc=np.size).reset_index()
features.append(f)

In [88]:
#아침점심저녁별 구매금액
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc="sum").reset_index()
features.append(f)

In [89]:
#주방문요일
f = tr.groupby('custid')['sales_dayofweek'].agg([('주방문요일', lambda x: x.value_counts().index[0])]).reset_index()
f = pd.get_dummies(f, columns=['주방문요일'])

features.append(f)

In [90]:
#평균 쇼핑시간
f = tr.groupby('custid')['total_sec'].agg([
    ('평균쇼핑시간', lambda x: (x.max() - x.min()) / x.nunique())]).reset_index()

features.append(f)

In [91]:
#일별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f)

In [92]:
#일별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f)

In [93]:
#월별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f)

In [94]:
#월별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f)

In [95]:
data = pd.concat([id_data,df_y_train],axis = 1)
data = data.iloc[:len(df_y_train)]

In [96]:
tr_data = pd.merge(tr,data,on="custid")

In [97]:
tr_cor_mean = dict(tr_data.groupby("corner_nm")["age"].mean())
tr_brd_mean = dict(tr_data.groupby("brd_nm")["age"].mean())
tr_pc_mean = dict(tr_data.groupby("pc_nm")["age"].mean())
tr_day_mean = dict(tr_data.groupby("sales_dayofweek")["age"].mean())

In [98]:
brd_data = tr.groupby("custid")["pc_nm"].agg([("pc","first")])

In [99]:
f = brd_data["pc"].apply(lambda x: tr_pc_mean[x])
features.append(f)

In [100]:
d = tr.groupby("custid")["corner_nm"].agg([("cor","first")])
f = d["cor"].apply(lambda x: tr_cor_mean[x])
features.append(f)

In [101]:
d = tr.groupby("custid")["sales_dayofweek"].agg([("day","first")])
f = d["day"].apply(lambda x: tr_day_mean[x])
features.append(f)

In [102]:
X_train = pd.DataFrame({'custid': df_x_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid')
display(X_train.shape)

X_test = pd.DataFrame({'custid': df_x_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid')

(21587, 715)

In [103]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train["주말방문비율"])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균구매가격'] / X_train['평균할부개월수']
X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

In [104]:
X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균구매가격'] / X_test['평균할부개월수']
X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

# kmean 클러스터링

In [105]:
features__ = X_train.drop(columns='custid',axis=1).fillna(0)
features_t__= X_test.drop(columns='custid',axis=1).fillna(0)

In [106]:
from sklearn.cluster import KMeans

class KMeansFeaturizer:
    """ 숫자 데이터를 k-평균 클러스터 멤버십으로 변환.

    이 변환기는 입력 데이터에 k-평균을 수행해 각 데이터 포인트를 가장 가까운 클러스터의 id로 변환한다.
    만약 목표 변수가 주어지면 유사한 데이터 포인트와 함께 grouping되고,
    분류 경계에 따르는 클러스터를 생성하기 위해 스케일링되고, k-평균 입력에 포함된다.
    """

    def __init__(self, k = 100, target_scale = 5.0, random_state = None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state

    def fit(self, X, y = None):
        """ 입력 데이터에 k-평균을 수행하고 중심점을 찾는다.
        """
        if y is None: # 목표 변수가 없으면 단순한 k-평균 수행
            km_model = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
            km_model.fit(X)
            
            self.inertia_ = km_model.inertia_
            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # 목표 변수가 있으면, 적절한 스케일링을 적용하고, 이를 k-평균에 대한 입력 데이터에 포함시킨다.
        data_with_target = np.hstack((X, y[:, np.newaxis] * self.target_scale))
        # 데이터와 타겟에 대해 사전 학습할 k-평균 모델 구축
        km_model_pretrain = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
        km_model_pretrain.fit(data_with_target)

        # k평균을 두번째로 실행해 목표 변수 없이 원시 공간에서 클러스터를 얻는다. 사전 학습을 통해 얻은 중심점을 활용해 초기화한다.
        # 반복을 통해 클러스터 할당과 중심점 계산을 다시 수행한다.

        km_model = KMeans(n_clusters = self.k, init = km_model_pretrain.cluster_centers_[:,:data_with_target.shape[1]-1], n_init = 1, max_iter = 1)

        km_model.fit(X)
        
        self.inertia_ = km_model.inertia_
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self

    def transform(self, X, y = None):
        """ 각 입력 데이터 포인트에 대해 가장 가까운 클러스터 ID 산출
        """
        clusters = self.km_model.predict(X)
        return clusters[:, np.newaxis]

    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return self.transform(X, y)

In [107]:
kme = KMeansFeaturizer(random_state = 100)

In [108]:
features_k = kme.fit_transform(features__)

In [109]:
features_k_t = kme.transform(features_t__)

In [110]:
features_kmean = pd.concat([pd.DataFrame(features_k),pd.DataFrame(features_k_t)],axis = 0).reset_index().drop("index",axis= 1)

In [111]:
features_kmean = features_kmean.astype("str")

In [112]:
features_k_dum = pd.get_dummies(features_kmean)

In [113]:
max_col = features_k_dum.shape[1]
pca = PCA(n_components=max_col, random_state=0).fit(features_k_dum)

cumsum = np.cumsum(pca.explained_variance_ratio_)
num_col = np.argmax(cumsum >= 0.99) + 1


if num_col == 1:
    num_col = max_col

pca = PCA(n_components = num_col, random_state=0).fit_transform(features_k_dum)
features_k_dum = pd.DataFrame(pca)

In [114]:
features_k_train = features_k_dum[:len(features__)]

In [115]:
features_k_test = features_k_dum[len(features__):].reset_index().drop("index",axis = 1)

In [116]:
features__ = pd.concat([features__,features_k_train],axis = 1)

In [117]:
features_t__ = pd.concat([features_t__,features_k_test],axis = 1)

In [118]:
identifier = features__.columns.to_series().groupby(level=0).transform('cumcount')
features__.columns = identifier.astype('string')

identifier_t = features_t__.columns.to_series().groupby(level=0).transform('cumcount')
features_t__.columns = identifier_t.astype('string')

In [119]:
identifier = features__.columns.to_series().groupby(level=0).transform('cumcount')
features__.columns = features__.columns.astype('string') + identifier.astype('string')

identifier_t = features_t__.columns.to_series().groupby(level=0).transform('cumcount')
features_t__.columns = features_t__.columns.astype('string') + identifier_t.astype('string')

# 1등 피쳐 스케일링

In [120]:
ji = pd.read_csv(os.path.abspath("../input")+"/지평오빠feature.csv")
ji_t = pd.read_csv(os.path.abspath("../input")+"/지평오빠feature_te.csv")

In [121]:
numeric_columns1 = ji.dtypes[ji.dtypes != 'object'].index.tolist()

In [122]:
skewlist1 = []
numeric_columns_ns1 = []

for i in numeric_columns1:
    if 'SKEW' in i:
        skewlist1.append(i)
    else:
        numeric_columns_ns1.append(i)

In [123]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ji[numeric_columns1] = scaler.fit_transform(ji[numeric_columns1])
ji_t[numeric_columns1] = scaler.transform(ji_t[numeric_columns1])

# 마이피쳐 스케일링

In [124]:
numeric_columns2 = features__.dtypes[features__.dtypes != 'object'].index.tolist()

In [125]:
skewlist2 = []
numeric_columns_ns2 = []

for i in numeric_columns2:
    if 'SKEW' in i:
        skewlist2.append(i)
    else:
        numeric_columns_ns2.append(i)

In [126]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features__[numeric_columns2] = scaler.fit_transform(features__[numeric_columns2])
features_t__[numeric_columns2] = scaler.transform(features_t__[numeric_columns2])


# 마이피쳐 + 1st 피쳐 병합

In [127]:
data = pd.concat([ji[numeric_columns_ns1],ji[skewlist1], features__[numeric_columns_ns2], features__[skewlist2] ], axis=1)

features_t__.index = ji_t.index
data_te = pd.concat([ji_t[numeric_columns_ns1],ji_t[skewlist1] , features_t__[numeric_columns_ns2], features_t__[skewlist2] ], axis=1)

# 워드투벡 병합

In [128]:
w2v_list = [w2v_good,w2v_coner,w2v_brd]

In [129]:
data_all = pd.concat([data,data_te],axis = 0)

In [130]:
data_all = data_all.reset_index().drop("index",axis = 1)

In [131]:
for i in w2v_list:
    data_all = pd.concat([data_all,i],axis = 1)

In [132]:
data_all.drop("custid",axis =1 ,inplace = True)

In [133]:
data = data_all[:len(features__)]
data_te = data_all[len(features__):].reset_index().drop("index",axis = 1)

In [134]:
data.columns = data.columns.astype(str)
data_te.columns = data_te.columns.astype(str)

In [135]:
data2 = data.values
data_te2 = data_te.values

In [136]:
ftr = data2[:, 1:]
target = df_y_train
target_log = np.log1p(target)

In [137]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.2 , stratify = target , random_state=1000)
train_x.shape, valid_x.shape

((17269, 17125), (4318, 17125))

In [138]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric = 'RMSE', 
        verbose=100, early_stopping_rounds= 50)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 7.7676	training's l2: 60.3357	valid_1's rmse: 8.30134	valid_1's l2: 68.9122
[200]	training's rmse: 7.02968	training's l2: 49.4164	valid_1's rmse: 8.09159	valid_1's l2: 65.4738
[300]	training's rmse: 6.53169	training's l2: 42.663	valid_1's rmse: 8.05721	valid_1's l2: 64.9186
[400]	training's rmse: 6.12835	training's l2: 37.5567	valid_1's rmse: 8.05002	valid_1's l2: 64.8028
Early stopping, best iteration is:
[406]	training's rmse: 6.10618	training's l2: 37.2855	valid_1's rmse: 8.04892	valid_1's l2: 64.7852


LGBMRegressor(learning_rate=0.02, max_depth=12, n_estimators=1000,
              num_leaves=32, silent=-1, subsample=0.8, verbose=-1)

In [139]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='7.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te2[:, 1:])
feature_idx = smf.get_support()
#feature_name = ftr.columns[feature_idx]

In [140]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_squared_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('RMSE:', np.sqrt(-np.mean(scores)))
# mean - 7.0 : RMSE: 7.99449713498378
# mean - 5.0 : RMSE: RMSE: 8.000763598952103

교차 검증별 정확도: [-63.5007 -65.8621 -65.4368 -61.9738 -62.0358]
평균 검증 정확도: -63.7618
RMSE: 7.985101113600758


In [141]:
#pd.DataFrame(X_new).to_csv("features__2rd_jinew.csv")
#pd.DataFrame(X_te_new).to_csv("features_t__2rd_jinew.csv")